In [2]:
from acevedo_clss_and_fcns import * 
from    cobra.io.mat import *
#import torch
from networkx.algorithms import bipartite
G = nx.read_gpickle("./results/graphs/NX_recon_graph.gpickle")
assert nx.is_connected(G)
assert 1 == np.unique(list(nx.get_edge_attributes(G, "weight").values())).__len__()
n_nodes                  = len(list(G.nodes))
partition_1, partition_2 = bipartite.sets(G)

In [ ]:


loader_path = './results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt'
loader = torch.load(loader_path)
a_batch         = next(iter(loader.get_train_loader()))
a_graph         = a_batch[0]
assert a_graph.x.shape[0] ==len(list(G.nodes))


In [13]:
feature_vector = a_graph.x.reshape(n_nodes).abs().tolist()

feature_dict   = dict(zip(G, feature_vector))
nx.set_node_attributes(G, feature_dict, "size")




In [15]:
nx.write_gexf(G, "./results/graphs/for_visualizations/recon3_pku.gexf")